In [26]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
import pandas as pd
import mne
data_dir = 'data_meg'
subj = "R2280"
dataqual = 'prepro' #or loc/exp
exp = 'loc' #or exp
dtype = "raw"
label_dir = 'data_log'
save_dir = 'data_meg'
bad_channels_dict = {
    "R2490": ['MEG 014', 'MEG 004', 'MEG 079', 'MEG 072', 'MEG 070', 'MEG 080', 'MEG 074', 'MEG 067', 'MEG 082', 'MEG 105', 'MEG 115', 'MEG 141', 'MEG 153'],
    "R2488": ['MEG 015', 'MEG 014', 'MEG 068', 'MEG 079', 'MEG 146', 'MEG 147', 'MEG 007', 'MEG 141'],
    "R2487": ['MEG 015', 'MEG 014', 'MEG 068', 'MEG 079', 'MEG 147', 'MEG 146', 'MEG 004'],
    "R2280": ['MEG 015', 'MEG 039', 'MEG 077', 'MEG 076', 'MEG 073', 'MEG 079', 'MEG 064', 'MEG 059', 'MEG 070']
}
bad_channels = bad_channels_dict.get(subj, [])


In [27]:
# raw = mne.io.read_raw_fif('data_meg/R2490/prepro/R2490_exp.fif', preload='temp_raw.fif')
raw = mne.io.read_raw_fif(f'{data_dir}/{subj}/{dataqual}/{subj}_{exp}.fif', preload=True)
raw.info['bads'].extend(bad_channels)
sfreq = raw.info['sfreq']
raw.filter(1, 40, method='iir')
downsample = 10
raw.resample(sfreq / downsample)
raw.drop_channels(bad_channels)

Opening raw data file data_meg/R2488/prepro/R2488_loc.fif...


/var/folders/79/m99pb_8d67l3d79st4zhf4tc0000gn/T/ipykernel_34489/591528618.py:2: RuntimeWarning: This filename (data_meg/R2488/prepro/R2488_loc.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(f'{data_dir}/{subj}/{dataqual}/{subj}_{exp}.fif', preload=True)


    Range : 0 ... 835999 =      0.000 ...   835.999 secs
Ready.
Reading 0 ... 835999  =      0.000 ...   835.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 40.00 Hz: -6.02, -6.02 dB

300 events found on stim channel STI 014
Event IDs: [160 164 165]
245 events found on stim channel STI 014
Event IDs: [160 164 165]


/var/folders/79/m99pb_8d67l3d79st4zhf4tc0000gn/T/ipykernel_34489/591528618.py:7: RuntimeWarning: Resampling of the stim channels caused event information to become unreliable. Consider finding events on the original data and passing the event matrix as a parameter.
  raw.resample(sfreq / downsample)


<Raw | R2488_loc.fif, 185 x 83600 (836.0 s), ~119.5 MB, data loaded>

In [28]:
events = mne.find_events(raw, stim_channel='STI 014', output='onset', shortest_event=1)
event_id = {
    'start': 160,
    'move': 161,
    'reveal_red': 162,
    'reveal_white': 163,
    'done': 164,
    'choice': 165,
    'timeout': 166
}

# Define trials to remove
trials_to_remove = []

245 events found on stim channel STI 014
Event IDs: [160 164 165]


In [29]:
start_events = events[events[:, 2] == event_id['start']]
done_events = events[events[:, 2] == event_id['done']] 
timeout_events = events[events[:, 2] == event_id['timeout']]
choice_events = events[events[:, 2] == event_id['choice']]
sfreq = raw.info['sfreq']

In [30]:
if subj == "R2488" :
    done_events = done_events[1:]

In [31]:

# Initialize a list to store filtered done events
filtered_done_events = [done_events[0]]  # Start with the first event

# Check for at least 3 seconds between each done event
for i in range(1, len(done_events)):
    time_diff = (done_events[i, 0] - done_events[i-1, 0]) / sfreq
    if time_diff < 2:
        print(f"Warning: Less than 3 seconds between done events at indices {i-1} and {i}")
    
    else:
        filtered_done_events.append(done_events[i])
done_events = filtered_done_events
# Use filtered_done_events for further processing
print(f"Filtered done events count: {len(filtered_done_events)}")



Filtered done events count: 120


In [38]:
choice_events = events[events[:, 2] == event_id['choice']]
start_events = np.array([[done_event[0] - int(3.4 * sfreq), 0, event_id['start']] for done_event in done_events])
# start_events = events[events[:, 2] == event_id['start']]
# start_events = start_events[-120:]
sfreq = raw.info['sfreq']  # Sampling frequency


# Initialize a list to store trial information
trial_info = []
start_idx = 0

# Iterate through each reconstructed start event to create trial information
for start_event, done_event in zip(start_events, done_events):
    start_sample = start_event[0]
    done_sample = done_event[0]
    # done_sample = start_sample + int(3.4 * sfreq)
    # Calculate tmin and tmax for the epoch
    tmin = -0.2  # 0.2 s before 'start'
    tmax = (done_sample - start_sample) / sfreq  # Duration from 'start' to 'done'
    # Find choice events within the trial
    choice_event = choice_events[(choice_events[:, 0] > start_sample) & 
                                 (choice_events[:, 0] < done_sample + int(0.5*sfreq))]
    choice_time = choice_event[-1, 0] if len(choice_event) > 0 else None
    
    # Store trial information
    trial_info.append({
        'event_sample': done_sample,
        'trial_index': start_idx,
        'duration': tmax,
        'tmin': tmin,
        'tmax': tmax,
        'done': len(done_events) > 0,
        'done_times': done_sample / sfreq,
        'start_times': start_sample / sfreq,
        'choice_event': len(choice_event) > 0,
        'choice_time': choice_time / sfreq if len(choice_event) > 0 else None
    })
    start_idx += 1

In [39]:
for info in trial_info:
    if info['choice_time'] is not None:
        print(info['trial_index'], info['choice_time'] - info['start_times'])
    elif info['done_times'] is not None:
        print(info['trial_index'], None )


0 1.8199999999999932
1 1.1099999999999852
2 1.079999999999984
3 0.7800000000000011
4 1.6799999999999784
5 1.3000000000000114
6 0.8100000000000023
7 2.1599999999999966
8 1.7800000000000011
9 0.7900000000000205
10 1.259999999999991
11 0.8199999999999932
12 1.0199999999999818
13 1.17999999999995
14 1.1300000000000523
15 0.6800000000000068
16 0.6499999999999773
17 1.0200000000000387
18 1.9800000000000182
19 1.150000000000034
20 1.349999999999966
21 0.75
22 0.8300000000000409
23 0.7599999999999909
24 0.7899999999999636
25 0.6899999999999977
26 2.419999999999959
27 0.8600000000000136
28 1.420000000000016
29 0.8500000000000227
30 1.349999999999966
31 1.420000000000016
32 0.8899999999999864
33 1.259999999999991
34 2.330000000000041
35 1.2200000000000273
36 2.2100000000000364
37 0.9300000000000068
38 1.660000000000025
39 0.9800000000000182
40 0.9499999999999886
41 0.7899999999999636
42 2.3899999999999864
43 0.9200000000000159
44 1.25
45 2.4600000000000364
46 0.9699999999999704
47 2.050000000000

In [40]:
new_events = np.array([[info['event_sample'], 0, event_id['done']] for info in trial_info])
tmin = -0.2  
# Initialize lists to store individual epochs data and trial information
data_list = []

for idx, event in enumerate(new_events):
    event_id_code = event[2]
    choice_time = trial_info[idx]['choice_time']  # Retrieve choice_time for the current trial
    print(choice_time)
    if choice_time is not None:
        start_sample = int(choice_time * sfreq)  # Convert choice_time to sample index
        tmin = -0.5  # 0.5 s before 'choice'
        tmax = 0.5  # 0.5 s after 'choice'
    else:
        done_sample = int(trial_info[idx]['done_times'] * sfreq)  # Convert done_times to sample index
        tmin = -1  # 1 s before 'done'
        tmax = 0  # At 'done'
    picks = mne.pick_types(raw.info, meg=True, exclude='bads')

    epochs = mne.Epochs(
        raw, [event], event_id={f'event_{event_id_code}': event_id_code},
        tmin=tmin, tmax=tmax, preload=True,picks=picks,
        reject_by_annotation=False, reject=None, verbose=True
    )
    data_list.append(epochs.get_data())

213.25
Not setting metadata


1 matching events found
Setting baseline interval to [-0.5, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 101 original time points ...
0 bad epochs dropped
217.48
Not setting metadata
1 matching events found
Setting baseline interval to [-0.5, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 101 original time points ...
0 bad epochs dropped
222.39
Not setting metadata
1 matching events found
Setting baseline interval to [-0.5, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 101 original time points ...
0 bad epochs dropped
227.04
Not setting metadata
1 matching events found
Setting baseline interval to [-0.5, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 101 original time points .

In [41]:
print(len(data_list))

120


In [36]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import joblib  # For saving the model

In [42]:
# Assuming X is your feature matrix with shape (n_epochs, n_channels, n_times)
X = np.array([md.data for md in data_list])  # Ensure this is 3D
X = X.squeeze(axis=1)  # This might reduce dimensions, ensure it's still 3D

# Load the CSV file
locolizer = pd.read_csv(f'{label_dir}/{subj}/loc_data.csv')

# Extract valid trial indices
valid_trial_indices = {info['trial_index'] for info in trial_info}

# Extract labels and group information
rule_label = locolizer.loc[locolizer['trial_index'].isin(valid_trial_indices), 'rule'].values
group_start = locolizer.loc[locolizer['trial_index'].isin(valid_trial_indices), 'num_start'].values

# Identify unique group_start labels
unique_groups = np.unique(group_start)

# Prepare data for each unique group_start
group_data = {group: [] for group in unique_groups}
group_labels = {group: [] for group in unique_groups}

# Populate group_data and group_labels
for idx, info in enumerate(trial_info):
    trial_index = info['trial_index']
    if trial_index in valid_trial_indices:
        group = group_start[idx]
        group_data[group].append(X[idx])  # Ensure X[idx] is 3D
        group_labels[group].append(rule_label[idx])

def flatten_data(X):
    n_samples, n_channels, n_timepoints = X.shape
    print(n_samples, n_channels, n_timepoints)
    return X.reshape(n_samples, n_channels * n_timepoints)

# Train and save decoders for each group
for group in unique_groups:
    X_group = np.array(group_data[group])
    y_group = np.array(group_labels[group])
    
    # Debugging: Check the shape of X_group
    print(f"Group: {group}, X_group shape: {X_group.shape}")
    # Flatten the data
    X_group_flat = flatten_data(X_group)
    
    # Train the decoder
    clf = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
    clf.fit(X_group_flat, y_group)
    
    # Save the trained model
    model_filename = f"locolizer/{subj}_decoder_{group}.joblib"
    joblib.dump(clf, model_filename)
    print(f"Model for group {group} saved as {model_filename}")

Group: 4, X_group shape: (30, 149, 101)
30 149 101
Model for group 4 saved as locolizer/R2488_decoder_4.joblib
Group: 9, X_group shape: (30, 149, 101)
30 149 101
Model for group 9 saved as locolizer/R2488_decoder_9.joblib
Group: 16, X_group shape: (30, 149, 101)
30 149 101
Model for group 16 saved as locolizer/R2488_decoder_16.joblib
Group: 25, X_group shape: (30, 149, 101)
30 149 101
Model for group 25 saved as locolizer/R2488_decoder_25.joblib
